In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, DateType, TimestampNTZType

## Link sensor data with questionnaire data

This is necessary for analyzing the statistical relationship between the sensor data and the questionnaire results.

**Steps**

1. Access the lookup table for the pseudonymization code generated by the app corresponding to the pseudonymization code related to the sensors;

2. Retrieve the questionnaires already completed by the subject (based on the pseudonymization code generated by the app)







In [0]:
# API call to get the questionnaire ID corresponding to the sensor ID
!curl --location 'https://ageit-test.azurewebsites.net/api/v1/users/devices/export' \
--header 'Content-Type: application/json' \
--header 'AGEIT-API-Key: w6dAwrDDoF9BR0VJVEJldGE4MDIwMjQqK8Op' \
--data '{}' \
-o /tmp/lookup_table.json # Save the response to a temp file

In [0]:
# Copy the JSON file to DBFS
dbutils.fs.cp("file:/tmp/lookup_table.json", "dbfs:/tmp/lookup_table.json")

# Read the JSON file as a Spark DataFrame
lookup_table = spark.read.json("dbfs:/tmp/lookup_table.json")

# Display the DataFrame
lookup_table.display()

In [0]:
# Device ID
sensor_ID = "14219"

# Get the questionnaire ID
questionnaire_ID = lookup_table.filter(lookup_table.deviceId == sensor_ID).select("userId").collect()[0][0]
questionnaire_ID

In [0]:
import pandas as pd
import os

path_quest_prod = "/dbfs/mnt/storageageittest/questionari-csv/UniqueFile_Output/"
months = os.listdir(path_quest_prod)

all_questions_mySub = pd.DataFrame()

for i, month in enumerate(months):
    days = os.listdir(path_quest_prod + month)
    for j, day in enumerate(days):
        quest = pd.read_csv(path_quest_prod + month + "/" + day, sep=";", encoding="utf-8")
        quest_mySub = quest[quest["UserId"] == questionnaire_ID]

        all_questions_mySub = pd.concat([all_questions_mySub, quest_mySub])

In [0]:
all_questions_mySub.sort_values(by="NomeQuestionario", inplace=True)
all_questions_mySub

In [0]:
all_questions_mySub[all_questions_mySub["Domanda"] == 'minacciareFarsiMaleReazione']["Descrizione"]